In [24]:
import pandas as pd
import numpy as np
import feather
import os as os
import glob
from joblib import Parallel, delayed
import multiprocessing
import gc

In [13]:
GFW_DIR = '/data2/GFW_point/'
GFW_OUT_DIR = '~/Data/GFW_point/Patagonia_Shelf/csv/'

In [5]:
gfw_vessel_dat = pd.read_csv('~/Data/GFW_public/fishing_vessels/fishing_vessels.csv')
dat = pd.read_csv('/data2/GFW_point/2016-01-01/messages-2016-01-01-000000000000.csv')
#gfw_identities = gfw_identities.rename(index=str, columns={"ssvid": "mmsi"})
#gfw_identities['year'] = pd.DatetimeIndex(gfw_identities['timestamp']).year 
#gfw_identities['month'] = pd.DatetimeIndex(gfw_identities['timestamp']).month
#gfw_identities['day'] = pd.DatetimeIndex(gfw_identities['timestamp']).day

/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def data_step(data): 
    # Patagonia shelf coordinates
    #lower left lat: -58
    #lower left lon: -77
    #upper right lat: -23
    #upper right lon: -22
    
    lon1 = -77
    lon2 = -22
    lat1 = -58
    lat2 = -23
    retdat = data[(data['lon'] >= lon1) & (data['lon'] <= lon2) & (data['lat'] >= lat1) & (data['lat'] <= lat2)]
    
    # Separate Year, month, day, hour, minute, second
    retdat.loc[:, 'timestamp'] = pd.to_datetime(retdat['timestamp'], format="%Y-%m-%d %H:%M:%S UTC")
    retdat.loc[:, 'year'] = pd.DatetimeIndex(retdat['timestamp']).year 
    retdat.loc[:, 'month'] = pd.DatetimeIndex(retdat['timestamp']).month
    retdat.loc[:, 'day'] = pd.DatetimeIndex(retdat['timestamp']).day
    retdat.loc[:, 'hour'] = pd.DatetimeIndex(retdat['timestamp']).hour
    retdat.loc[:, 'minute'] = pd.DatetimeIndex(retdat['timestamp']).minute
    retdat.loc[:, 'second'] = pd.DatetimeIndex(retdat['timestamp']).second
    
    # Merge GFW ID data
    retdat = pd.merge(retdat, gfw_vessel_dat, how='left', on='mmsi')  
    
    retdat = retdat[['timestamp', 'year', 'month', 'day', 'hour', 'minute', 'second', 'mmsi', 'lat', 'lon', \
                    'segment_id', 'message_id', 'type', 'speed', 'course', 'heading', 'shipname', 'callsign', \
                     'destination', 'elevation_m', 'distance_from_shore_m', 'distance_from_port_m', 'nnet_score', \
                     'logistic_score', 'flag', 'geartype', 'length', 'tonnage', 'engine_power', 'active_2012', \
                     'active_2013', 'active_2014', 'active_2015', 'active_2016']]
    
    return retdat

In [26]:
def GFW_directories(directory):
    
    dirs = os.listdir(directory)
    # Remove subfolders 'BK' and 'identities'
    if 'BK' in dirs:
        dirs.remove('BK')
    
    if 'identities' in dirs:
        dirs.remove('identities')
    
    return dirs

In [8]:
ndat = data_step(data=dat)

/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [28]:
dirs = sorted(GFW_directories(GFW_DIR))
len(dirs)

1096

In [25]:
allFiles

NameError: name 'allFiles' is not defined

In [7]:
dirs = sorted(GFW_directories(GFW_dir))

for i in dirs[1:2]:  # Change!!!

    # Get subdirectory list of files
    subdir = GFW_DIR + i
    allFiles = glob.glob(subdir + "/*.csv")
    list_ = []
    
    # Append files in subdir
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0)
        list_.append(df)
        dat = pd.concat(list_, axis = 0, ignore_index = True)
    
    # Append data
    outdat = data_step(data=dat)
    outdat = outdat.fillna()

    # Get string for filename from timestamp
    filename = f"{outdat['year'][1]}-" + f"{outdat['month'][1]}".zfill(2) + f"-" + f"{outdat['day'][1]}".zfill(2)
    
    # Save unique mmsi for each day
    unique_mmsi_data = outdat['mmsi'].unique()
    unique_mmsi = pd.DataFrame({'mmsi':unique_mmsi_data})
    unique_mmsi.to_feather('~/Data/GFW_point/Patagonia_Shelf/vessel_list/' + filename +  '_vessel_list'  + '.feather')
    
    # Save data
    outdat.to_csv('~/Data/GFW_point/Patagonia_Shelf/csv/' + filename + '.csv', index=False)
    outdat.to_feather('~/Data/GFW_point/Patagonia_Shelf/feather/' + filename + '.feather')
    print(i)

/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


ValueError: Must specify a fill 'value' or 'method'.

In [232]:
inputs = dirs
def processGFW(i):
    subdir = GFW_DIR + i
    allFiles = glob.glob(subdir + "/*.csv")
    list_ = []
    
    # Append files in subdir
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0)
        list_.append(df)
        dat = pd.concat(list_, axis = 0, ignore_index = True)
    outdat = data_step(data=dat, filename=i)
    outdat.to_csv('~/Data/GFW_point/Patagonia_Shelf/csv/' + i + '.csv')
    outdat.to_feather('~/Data/GFW_point/Patagonia_Shelf/feather/' + i + '.feather')
    #print(i)  
 
num_cores = 20
results = Parallel(n_jobs=num_cores)(delayed(processGFW)(i) for i in inputs)